In [194]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [195]:
file = 'source.xlsx'
x1 = pd.ExcelFile(file)
x1.sheet_names
df = x1.parse('months')

In [196]:
# список департаментов первого уровня (territ_subdiv_1)
territ_subdiv_1_list = list(set(df['territ_subdiv_1']))

In [206]:
def predict_func(df, start):
    """
    df -- дата-фрэйм, подготовленный для модели
    start -- дата начала прогноза
    return: таблица со значениями, тип - pandas.Series
    """
    p = d = q = range(0, 2)
    # Генерируем различные комбинации p, q и q
    pdq = list(itertools.product(p, d, q))
    # Генерируем комбинации сезонных параметров P, D и Q, где 12 - периодичность временного ряда
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))] 
    
    warnings.filterwarnings("ignore") # отключает предупреждения
    
    res_aic = 0 # переменная для хранения наилучшего результата для модели
    
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(df,
                order=param,
                seasonal_order=param_seasonal,
                enforce_stationarity=False,
                enforce_invertibility=False)
                results = mod.fit()
#                 print(f'ARIMA{param}x{param_seasonal} - AIC:{results.aic}')
                if results.aic < res_aic:    # если текущий коэффициент AIC меньше предыдущего
                    res_aic = results.aic    # сохраняем этот AIC
                    res_param = param        # сохраняем кортеж с p, d, q
                    res_param_seasonal = param_seasonal   # сохраняем кортеж с P, D, Q, S
                else:
                    continue
            except:
                continue
                
    mod = sm.tsa.statespace.SARIMAX(
        df,
        order=(res_param[0], res_param[1], res_param[2]),
        seasonal_order=(res_param_seasonal[0], res_param_seasonal[1], res_param_seasonal[2], res_param_seasonal[3]),
        enforce_stationarity=False,
        enforce_invertibility=False)
    results = mod.fit()
    
    # Сравним прогнозируемые значения с реальными значениями временного ряда, 
    # что поможет нам понять точность прогнозов. 
    # Атрибуты get_prediction () и conf_int () позволяют получать значения и интервалы для прогнозов временных рядов.
    pred = results.get_prediction(start=pd.to_datetime(start), dynamic=False)
    pred_ci = pred.conf_int()
    
    # Вычислить среднеквадратичную ошибку
    y_forecasted = pred.predicted_mean
    y_truth = df[start:].squeeze() 
    mse = ((y_forecasted - y_truth) ** 2).mean()
    print(f'The Mean Squared Error of our forecasts is {round(mse, 2)}')
    
    # Получить прогноз на 12 шагов вперёд
    pred_uc = results.get_forecast(steps=12)
    
    # Получить интервал прогноза
    pred_ci = pred_uc.conf_int()
    
    print('Task is complete. The result is received!')
    print('*************')
    
    return pred_uc.predicted_mean

In [207]:
# создаем пустой DataFrame 
res_df = pd.DataFrame()
for el1 in territ_subdiv_1_list:
    # проходим в цикле все департаменты первого уровня
    df_query = df.query('territ_subdiv_1 == @el1')
    # находим все подразделения второго урвоня (territ_subdiv_2), на которые дробится первый уровень
    territ_subdiv_2_list = list(set(df_query['territ_subdiv_2']))
    for el2 in territ_subdiv_2_list:
        print(f'Current territ_subdiv_1: {el1} and Current territ_subdiv_2: {el2}')
        # выбираем из DF только нужные столбцы ('report_dt и val'), сбрасываем индексы, готовим DF для работы с функцией 
        # предсказания временного ряда
        df1 = df.query('territ_subdiv_1 == @el1 and territ_subdiv_2 == @el2')[['report_dt', 'val']] \
                .set_index('report_dt')
        # запускаем функцию предсказания и делаем DF из pandas.Series
        temp_df = predict_func(df=df1, start='2018-01-31').to_frame('val')
        # добавляем столбцы с территориями
        temp_df = temp_df.reset_index() \
                        .assign(territ_subdiv_1=el1, territ_subdiv_2=el2) \
                        .rename(columns={'index': 'report_dt'}) \
                            [['report_dt', 'territ_subdiv_1', 'territ_subdiv_2', 'val']]
        # на каждом шаге добавляем к существующим данным новый DF
        res_df = res_df.append(temp_df)
        
# записываем результат в файл Excel
print('Запись в файл Excel...')
with pd.ExcelWriter('output.xlsx', datetime_format='DD.MM.YYYY', mode='a', engine='openpyxl') as writer:  
            res_df.to_excel(writer, index=False, header=True)
print('Работа завершена!')

Current territ_subdiv_1: 1 and Current territ_subdiv_2: 1
The Mean Squared Error of our forecasts is 0.01
Task is complete. The result is received!
*************
Current territ_subdiv_1: 1 and Current territ_subdiv_2: 2
The Mean Squared Error of our forecasts is 0.01
Task is complete. The result is received!
*************
Current territ_subdiv_1: 2 and Current territ_subdiv_2: 1
The Mean Squared Error of our forecasts is 0.0
Task is complete. The result is received!
*************
Current territ_subdiv_1: 2 and Current territ_subdiv_2: 2
The Mean Squared Error of our forecasts is 0.0
Task is complete. The result is received!
*************
Запись в файл Excel...
Работа завершена!
